In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random

import sys
sys.path.append("Fall 2024")

import kmax as kx
import utility as util
from hierarchy import *

# Continuous Edge Weights - Transition Probabilities

## Function Definitions

## Simulation

In [2]:
num_samples = 1000
lam = 0.7
offspring_dist = lambda: np.random.poisson(lam)
edge_dist = lambda: np.random.uniform(0,1)

In [6]:
for _ in range(num_samples):
    # Generate tree
    tree = nx.Graph(GWBP(dist=offspring_dist, MAXLEVEL=10))
    for u,v in tree.edges():
        tree[u][v]['weight'] = edge_dist()
    
    # Get Z values
    # TODO - this doesn't work in a really weird way...
    tups = util.path_cover(tree, root=0, return_tuples=True)
    #Z = {e:tup[1] for e,tup in tups}
    Z = {e:tup[1] for e,tup in tups.items()}

{3: (0, 0.9735632083291947, None, None), 2: (0.9735632083291947, 0.5149527263497861, 3, None), 1: (1.488515934678981, 0.9534208564580703, 2, None), 0: (2.441936791137051, 1, 1, None)}


AttributeError: 'tuple' object has no attribute 'items'

In [ ]:
type(tups)

In [5]:
for x in tups:
    print(x)

3
8
9
7
6
5
4
2
1
0
